In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score

# Đọc dữ liệu từ các tệp CSV riêng biệt
df_train = pd.read_csv("dataset/train.csv")
# df_val = pd.read_csv("dataset/validation.csv")
df_test = pd.read_csv("dataset/test.csv")

# Loại bỏ cột 'video_name'
df_train = df_train.drop(columns=["video_name"])
# df_val = df_val.drop(columns=["video_name"])
df_test = df_test.drop(columns=["video_name"])

# Các nhãn bài tập (được đánh số từ 0 đến 5)
label_column = "label"

# Tách feature và label
X_train = df_train.drop(columns=[label_column]).values
y_train = df_train[label_column].values

# X_val = df_val.drop(columns=[label_column]).values
# y_val = df_val[label_column].values

X_test = df_test.drop(columns=[label_column]).values
y_test = df_test[label_column].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Train mô hình Logistic Regression
model = LogisticRegression(max_iter=1000, multi_class='ovr')  # One-vs-Rest cho multi-label
model.fit(X_train, y_train)

# Đánh giá trên tập validation
# y_val_pred = model.predict(X_val)
# val_accuracy = accuracy_score(y_val, y_val_pred)
# val_recall = recall_score(y_val, y_val_pred, average='macro')
# print(f'Validation Accuracy: {val_accuracy:.4f}')
# print(f'Validation Recall: {val_recall:.4f}')

# Đếm số lượng mẫu của mỗi lớp
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
print('Class counts:', class_counts)


# Tạo trọng số tùy chỉnh
# class_weight = {3: 2, 4: 3, 5: 3}  # Giảm trọng số lớp 0, tăng lớp 3, 4, 5
# for cls in unique:  
#     if cls not in class_weight:  
#         class_weight[cls] = 1  # Các lớp khác giữ nguyên trọng số

# Train mô hình với class_weight
model = LogisticRegression(max_iter=1000, multi_class='ovr')
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall_per_class = recall_score(y_test, y_pred, average=None)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Recall:')
for i, recall in enumerate(recall_per_class):
    print(f'Class {i}: {recall:.4f}')

d:\Thanh\TLHT\HK6\PBL5\Main\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Class counts: {0: 3150, 1: 2447, 2: 2495, 3: 1316, 4: 927, 5: 805}


d:\Thanh\TLHT\HK6\PBL5\Main\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Test Accuracy: 0.8740
Test Recall:
Class 0: 0.7765
Class 1: 0.7933
Class 2: 1.0000
Class 3: 0.8955
Class 4: 0.9398
Class 5: 0.9890


In [20]:
import pickle


# Lưu mô hình vào file .pkl
with open('Model/Squat_detection_LR.pkl', 'wb') as f:
    pickle.dump(model, f)

with open("Model/scaler_LR.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [21]:
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Danh sách keypoints quan trọng
IMPORTANT_KP = [
    "NOSE", 
    "LEFT_SHOULDER", 
    "RIGHT_SHOULDER", 
    "LEFT_HIP", 
    "RIGHT_HIP", 
    "LEFT_KNEE", 
    "RIGHT_KNEE", 
    "LEFT_ANKLE", 
    "RIGHT_ANKLE"
]

# Load mô hình và bộ chuẩn hóa
with open("Model/Squat_detection_LR.pkl", "rb") as f:
    model = pickle.load(f)
with open("Model/scaler_LR.pkl", "rb") as f:
    scaler = pickle.load(f)

# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Đọc video đầu vào
video_path = "Data/Test/Correct/20250228_081403000_iOS (video-converter.com).mp4"  # Thay bằng đường dẫn video của bạn
cap = cv2.VideoCapture(video_path)

# Lấy thông tin video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Tạo VideoWriter để lưu video đầu ra
output_path = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Định dạng MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Thoát nếu hết video
    
    # Chuyển đổi sang RGB để xử lý với MediaPipe
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Lấy tọa độ x, y, z và độ tin cậy của keypoints quan trọng
        features = []
        for kp in IMPORTANT_KP:
            landmark = getattr(mp_pose.PoseLandmark, kp)
            features.extend([landmarks[landmark].x, landmarks[landmark].y, landmarks[landmark].z, landmarks[landmark].visibility])
        
        # Chuyển đổi thành numpy array và chuẩn hóa
        features = np.array(features).reshape(1, -1)
        features = scaler.transform(features)
        
        # Dự đoán bằng model
        prediction_probs = model.predict_proba(features)  # Trả về mảng xác suất
        label = np.argmax(prediction_probs)  # Lấy nhãn có xác suất cao nhất
        
        # Kiểm tra xác suất dự đoán
        print("Prediction probabilities:", prediction_probs)
        print("Predicted label:", label)

        
        # Nhãn lỗi Squat
        labels_dict = {
            0: "Correct",
            1: "Chan qua hep",
            2: "Chan qua rong ",
            3: "Goi qua gan ",
            4: "Xuong qua sau ",
            5: "Gap lung"
        }
        label_text = labels_dict.get(label + 1, "Unknown")
        
        # Hiển thị nhãn lên video
        cv2.putText(frame, f"Prediction: {label_text}", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    
    # Ghi frame có nhãn vào video output
    out.write(frame)
    
    # Hiển thị video trong quá trình xử lý
    cv2.imshow("Squat Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()


Prediction probabilities: [[5.00442954e-04 1.66780496e-02 2.85682351e-05 9.82771149e-01
  7.31220877e-21 2.17901951e-05]]
Predicted label: 3
Prediction probabilities: [[1.05268579e-03 1.19546278e-02 3.72964185e-05 9.86947797e-01
  4.61706786e-21 7.59336522e-06]]
Predicted label: 3
Prediction probabilities: [[1.24418286e-03 9.82252773e-03 5.68186666e-05 9.88872879e-01
  3.47186743e-21 3.59222688e-06]]
Predicted label: 3
Prediction probabilities: [[1.40998567e-03 9.73806086e-03 5.16340269e-05 9.88798073e-01
  3.05545285e-21 2.24613473e-06]]
Predicted label: 3
Prediction probabilities: [[9.43758808e-04 1.08571868e-02 4.57908520e-05 9.88148752e-01
  6.15142869e-21 4.51166578e-06]]
Predicted label: 3
Prediction probabilities: [[7.66670209e-04 1.21168820e-02 4.45766219e-05 9.87065852e-01
  8.09550142e-21 6.01943169e-06]]
Predicted label: 3
Prediction probabilities: [[6.26674458e-04 1.27442071e-02 4.09910953e-05 9.86580438e-01
  1.08176614e-20 7.68954514e-06]]
Predicted label: 3
Prediction pr